### Importing Libraries

In [1]:
import numpy as np

import pandas as pd

import json #library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests
import urllib.request

from bs4 import BeautifulSoup
from urllib.request import urlopen

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# 1. DATA COLLECTION

# Toronto Data

First we scrape the Dataset of Torronto's neighbourhoods from a wikipedia link.

Toronto has a total of 10 boroughs and 103 neighborhoods.

We segment the neighborhoods and explore them, we will essentially need a dataset that contains the 10 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

#### Data Scraping

In [2]:
#using beatiful soup library to scrape data from a wikipedia page

url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [3]:
#storing the table in a pandas data frame

all_tab=soup.find_all('table')
right_table=soup.find('table',class_='wikitable sortable')

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

toronto_df=pd.DataFrame(A,columns=['PostCode'])
toronto_df['Borough']=B
toronto_df['Neighborhood']=C

We get the scraped data as follows :

In [4]:
toronto_df.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Cleaning the data set

In [5]:
#we drop the rows where Borough is "Not assigned"

toronto_df=toronto_df[toronto_df["Borough"]!='Not assigned']
toronto_df.reset_index(inplace=True)
toronto_df.drop('index',axis=1,inplace=True)
toronto_df.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [6]:
#Combining rows where more than one neighborhood exists in one postal code area 
#into one row with the neighborhoods separated with a comma 

toronto_df['Neighborhood'] = toronto_df.groupby(['PostCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
toronto_df.drop_duplicates(inplace=True)
toronto_df.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [7]:
toronto_df.rename(columns={'PostCode':'Postal Code'},inplace=True)

In [8]:
#having a look at the boroughs in Toronto
toronto_df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

#### Adding the longitude and Latitude values

In [9]:
#loading the longitude and latitude data into a pandas data frame
longlat_toronto=pd.read_csv("http://cocl.us/Geospatial_data")
longlat_toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#merging the two dataframes
toronto_df=pd.merge(toronto_df,longlat_toronto,on='Postal Code')

### Thus we get the final dataset for toronto

In [11]:
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


Making sure that the dataset has all 10 boroughs and 103 neighborhoods.

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


# NYC Data

NYC has a total of 5 boroughs and 306 neighborhoods. 

In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web in form of a .json file

#### Fetching Data

In [13]:
#loading the json file into a python variable and dropping the irrelevant information

url="https://cocl.us/new_york_dataset"
filename = 'newyork_data.json'
urllib.request.urlretrieve(url, filename)

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

nyc_data = newyork_data['features']

In [14]:
# Tranforming the data into a pandas dataframe

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
nyc_df = pd.DataFrame(columns=column_names)

nyc_df

,Borough,Neighborhood,Latitude,Longitude


In [15]:
# looping through the data and fill the dataframe one row at a time.

for data in nyc_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_df = nyc_df.append({'Borough': borough,
                            'Neighborhood': neighborhood_name,
                            'Latitude': neighborhood_lat,
                            'Longitude': neighborhood_lon}, ignore_index=True)

### Thus we get the final dataset for New York City

In [16]:
nyc_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Making sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nyc_df['Borough'].unique()),
        nyc_df.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Now to we have 103 neighbourhoods in Torronto and 306 neighbourhoods in NYC, let's subset Manhattan (a.k.a. New York)'s data also

### And thus the dataset for Manhattan (New York)

In [18]:
manhattan_data = nyc_df[nyc_df['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


# 2. DATA VISUALIZATION

# Toronto

### Creating a map of Toronto with neighborhoods superimposed on top.

In [19]:
# Using geopy library to get the latitude and longitude values of Toronto.

address_t = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location_t = geolocator.geocode(address_t)
latitude_t = location_t.latitude
longitude_t = location_t.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_t, longitude_t))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
map_toronto = folium.Map(location=[latitude_t, longitude_t], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# New York City (and New York)

### Creating a map of New York City with neighborhoods superimposed on top.

In [22]:
# Using geopy library to get the latitude and longitude values of Toronto.

address_n = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location_n = geolocator.geocode(address_n)
latitude_n = location_n.latitude
longitude_n = location_n.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_n, longitude_n))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [23]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_n, longitude_n], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nyc_df['Latitude'], nyc_df['Longitude'], nyc_df['Borough'], nyc_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Let's also get the geographical coordinates of Manhattan.

In [24]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualize Manhattan and the neighborhoods in it.

In [25]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

# 3. FETCHING LOCATION DATA (FOURSQUARE API)

#### Defining Foursquare Credentials and Version

In [26]:
CLIENT_ID = '1EH01SK1KF05C4GKBFLMWEPRE2EWBS10TKXQE4X3DQUELTFA' # your Foursquare ID
CLIENT_SECRET = 'VJFZ4UL3PP0ZHZS3G4IFIVTBSSUSM5QIFLTMLMPJYAJEMRWG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1EH01SK1KF05C4GKBFLMWEPRE2EWBS10TKXQE4X3DQUELTFA
CLIENT_SECRET:VJFZ4UL3PP0ZHZS3G4IFIVTBSSUSM5QIFLTMLMPJYAJEMRWG


<a id='item2'></a>

### Exploring Neighborhoods 

We create a function to explore all the neighborhoods

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Toronto

In [28]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude'],
                                  )



Parkwoods
Victoria Village
Harbourfront
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North

Woodbine Gardens,Parkview Hill
Ryerson
,Garden District

Glencairn

Cloverdale
,Islington,Martin Grove
,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
,Eringate
,Markland Wood,Old Burnhamthorpe

Guildwood
,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor,Downsview North
,Wilson Heights
Thorncliffe Park
Adelaide
,King
,Richmond

Dovercourt Village,Dufferin

Scarborough Village
Fairview
,Henry Farm,Oriole

Northwood Park,York University
East Toronto
Harbourfront East
,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park
,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East

The Danforth West
,Rive

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(2185, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Shape of this dataframe tells us that there are 2188 venues in Toronto

Now, Checking how many venues were returned for each neighborhood

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide\n,King\n,Richmond\n",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East\n,Milliken,Steeles East\n",2,2,2,2,2,2
"Albion Gardens\n,Beaumond Heights,Humbergate\n,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",8,8,8,8,8,8
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North\n,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East\n",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 264 uniques categories.


<a id='item3'></a>

# New York City (and New York)

In [32]:
nyc_venues = getNearbyVenues(names=nyc_df['Neighborhood'],
                                   latitudes=nyc_df['Latitude'],
                                   longitudes=nyc_df['Longitude'],
                                  )



Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [33]:
print(nyc_venues.shape)
nyc_venues.head()

(10259, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


#### Shape of this dataframe tells us that there are 10259 venues in New York City
#### Clearly comparing 2188 venues to 10259 venues doesn't make sense.
#### Thus, let's go ahead with comparing Manhattan's Neighbourhoods with those of Toronto instead of NYC

Now we see that we have 

In [34]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [35]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3297, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


#### Shape of this dataframe tells us that there are 3297 venues in Manhattan (New York)

#### 3297 venues are comparable to 2188 venues.

#### Thus it only makes sense to using Manhattan's dataset ahead instead of New York City.



In [36]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,89,89,89,89,89,89
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


In [37]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 343 uniques categories.


# CLUSTERING

# Toronto

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_onehot.shape

(2185, 264)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide\n,King\n,Richmond\n",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.01
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North,L'Amoreaux East\n,Milliken,Ste...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Albion Gardens\n,Beaumond Heights,Humbergate\n...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Bathurst Manor,Downsview North\n,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,"Bedford Park,Lawrence Manor East\n",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,"Birch Cliff,Cliffside West\n",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [41]:
toronto_grouped.shape

(100, 264)

#### Printing each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
,King
,Richmond
----
         venue  freq
0  Coffee Shop  0.07
1   Restaurant  0.06
2         Café  0.05
3          Bar  0.03
4       Bakery  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4              Metro Station  0.00


----Agincourt North,L'Amoreaux East
,Milliken,Steeles East
----
               venue  freq
0         Playground   0.5
1               Park   0.5
2        Yoga Studio   0.0
3        Men's Store   0.0
4  Mobile Phone Shop   0.0


----Albion Gardens
,Beaumond Heights,Humbergate
,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.25
1            Beer Store  0.12
2   Fried Chicken Joint  0.12
3  Fast Food Restaurant  0.12
4              Pharmacy  0.12


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.22
1      

              venue  freq
0  Department Store   0.2
1       Bus Station   0.2
2    Discount Store   0.2
3       Coffee Shop   0.2
4        Hobby Shop   0.2


----East Toronto----
               venue  freq
0               Park  0.33
1        Coffee Shop  0.33
2  Convenience Store  0.33
3        Yoga Studio  0.00
4      Metro Station  0.00


----Emery,Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2  Molecular Gastronomy Restaurant   0.0
3               Mac & Cheese Joint   0.0
4                           Market   0.0


----Fairview
,Henry Farm,Oriole
----
                  venue  freq
0        Clothing Store  0.13
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.07
3   Japanese Restaurant  0.05
4         Women's Store  0.03


----First Canadian Place,Underground city----
              venue  freq
0       Coffee Shop  0.12
1              Café  0.07
2        Restaurant  0.06
3             Ho

                venue  freq
0         Pizza Place  0.08
1    Sushi Restaurant  0.08
2         Coffee Shop  0.08
3                Café  0.08
4  Italian Restaurant  0.05


----Ryerson
,Garden District
----
                 venue  freq
0          Coffee Shop  0.09
1       Clothing Store  0.09
2  Japanese Restaurant  0.03
3                 Café  0.03
4      Bubble Tea Shop  0.03


----Scarborough Village----
               venue  freq
0         Playground   0.5
1  Convenience Store   0.5
2        Yoga Studio   0.0
3        Men's Store   0.0
4  Mobile Phone Shop   0.0


----Silver Hills
,York Mills----
                venue  freq
0           Cafeteria   1.0
1         Yoga Studio   0.0
2  Mac & Cheese Joint   0.0
3              Market   0.0
4   Martial Arts Dojo   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4               Hotel  0.04


----Stn A PO Boxes 25 The Esplan

#### Putting that into a *pandas* dataframe

We write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new dataframe and display the top 10 venues for each neighborhood.

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide\n,King\n,Richmond\n",Coffee Shop,Restaurant,Café,Thai Restaurant,Bakery,Bar,Concert Hall,Vegetarian / Vegan Restaurant,Sushi Restaurant,Office
1,Agincourt,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,"Agincourt North,L'Amoreaux East\n,Milliken,Ste...",Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens\n,Beaumond Heights,Humbergate\n...",Grocery Store,Beer Store,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Pizza Place,Pharmacy,Donut Shop,Doner Restaurant,Dog Run
4,"Alderwood,Long Branch",Pizza Place,Pub,Coffee Shop,Gym,Skating Rink,Pharmacy,Athletics & Sports,Sandwich Place,Diner,Deli / Bodega


<a id='item4'></a>

Running *k*-means to cluster the neighborhood into 5 clusters.

In [45]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged=toronto_merged.dropna()

toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,BBQ Joint,Food & Drink Shop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Portuguese Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Bakery,Distribution Center
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Accessories Store,Gift Shop
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Bank,Beer Bar,Seafood Restaurant,Boutique,Burger Joint,Burrito Place,Café,College Auditorium


### Final Visualisation of clusters

Finally, let's visualize the resulting clusters

<a id='item5'></a>

In [49]:
# create map
map_clusters = folium.Map(location = [latitude_t, longitude_t], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Portuguese Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,0,Coffee Shop,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Bakery,Distribution Center
3,North York,0,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Accessories Store,Gift Shop
4,Downtown Toronto,0,Coffee Shop,Yoga Studio,Bank,Beer Bar,Seafood Restaurant,Boutique,Burger Joint,Burrito Place,Café,College Auditorium
6,Scarborough,0,Fast Food Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
7,North York,0,Café,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dance Studio
8,East York,0,Pizza Place,Fast Food Restaurant,Breakfast Spot,Bank,Gym / Fitness Center,Pharmacy,Gastropub,Athletics & Sports,Intersection,Bus Line
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Lingerie Store,Restaurant
10,North York,0,Park,Sushi Restaurant,Pub,Pizza Place,Japanese Restaurant,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
11,Etobicoke,0,Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


#### Cluster 2

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,BBQ Joint,Food & Drink Shop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
21,York,1,Park,Women's Store,Market,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
35,East York,1,Park,Convenience Store,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
40,North York,1,Park,Airport,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
49,North York,1,Park,Construction & Landscaping,Bakery,Basketball Court,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Department Store,Doner Restaurant
61,Central Toronto,1,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Empanada Restaurant
64,York,1,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
66,North York,1,Park,Bar,Bank,Convenience Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Department Store,Dog Run
68,Central Toronto,1,Trail,Park,Jewelry Store,Sushi Restaurant,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
85,Scarborough,1,Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


#### Cluster 3

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,2,Convenience Store,Playground,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,Central Toronto,2,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


#### Cluster 4

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3,Baseball Field,Food Truck,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Ethiopian Restaurant
57,North York,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
101,Etobicoke,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 5

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Cafeteria,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


# New York (Manhattan)

In [55]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
manhattan_onehot.shape

(3297, 344)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.00,0.00,0.000000,0.011236,0.00,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.033708,0.000000,0.022472,0.000000
1,Carnegie Hill,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.00,0.000000,...,0.020000,0.00,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.010000,0.030000
2,Central Harlem,0.000000,0.00,0.00,0.069767,0.046512,0.00,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.000000,0.010000,0.000000
4,Chinatown,0.000000,0.00,0.00,0.000000,0.040000,0.00,0.000000,0.00,0.000000,...,0.030000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.010000
5,Civic Center,0.000000,0.00,0.00,0.000000,0.030000,0.01,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.020000,0.010000,0.000000,0.030000
6,Clinton,0.000000,0.00,0.00,0.000000,0.040000,0.00,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.030000,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.01,0.010000,...,0.020000,0.00,0.000000,0.00,0.00,0.040000,0.010000,0.000000,0.000000,0.000000
9,Financial District,0.010000,0.00,0.00,0.000000,0.050000,0.00,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.010000,0.000000


In [58]:
manhattan_grouped.shape

(40, 344)

#### Printing each neighborhood along with the top 5 most common venues

In [59]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
         venue  freq
0  Coffee Shop  0.07
1         Park  0.07
2        Hotel  0.06
3          Gym  0.03
4    Wine Shop  0.03


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.06
1  Pizza Place  0.06
2         Café  0.05
3  Yoga Studio  0.03
4       Bakery  0.03


----Central Harlem----
                venue  freq
0  African Restaurant  0.07
1  Seafood Restaurant  0.05
2   French Restaurant  0.05
3  Chinese Restaurant  0.05
4                 Bar  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.07
1               Bakery  0.06
2   Italian Restaurant  0.05
3  American Restaurant  0.03
4            Wine Shop  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.05
2  American Restaurant  0.04
3               Bakery  0.03
4         Optical Shop  0.03


----Civic Center----
                  venue  freq
0           Coffee Shop  0.06
1  Gym / Fitness Center  0.05
2     

#### Putting that into a *pandas* dataframe

We write a function to sort the venues in descending order.

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and display the top 10 venues for each neighborhood.

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Boat or Ferry,Memorial Site,Shopping Mall,Gym,Italian Restaurant,Food Court
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Japanese Restaurant,Gym / Fitness Center,Gym,French Restaurant,Cosmetics Shop,Bookstore
2,Central Harlem,African Restaurant,Bar,French Restaurant,American Restaurant,Seafood Restaurant,Chinese Restaurant,Caribbean Restaurant,Spa,Dessert Shop,Beer Bar
3,Chelsea,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Ice Cream Shop,Hotel,Wine Shop,Breakfast Spot,Tapas Restaurant,Cycle Studio
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Bakery,Hotpot Restaurant,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop


<a id='item4'></a>

Running *k*-means to cluster the neighborhood into 5 clusters.

In [62]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 2, 0, 1, 1])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Bakery,Hotpot Restaurant,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Latin American Restaurant,Deli / Bodega,Tapas Restaurant,New American Restaurant,Park
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Restaurant,Café,Pizza Place,Wine Bar,American Restaurant,Bakery,Frozen Yogurt Shop,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Sandwich Place,Indian Restaurant,Sushi Restaurant,Park,Deli / Bodega,Yoga Studio


### Final Visualization of clusters

Finally, let's visualize the resulting clusters

In [66]:
# create map
map_clusters = folium.Map(location=[latitude_n, longitude_n], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

#### Cluster 1

In [67]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[0,1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Manhattan,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Latin American Restaurant,Deli / Bodega,Tapas Restaurant,New American Restaurant,Park
3,Manhattan,Inwood,Mexican Restaurant,Lounge,Restaurant,Café,Pizza Place,Wine Bar,American Restaurant,Bakery,Frozen Yogurt Shop,Deli / Bodega
4,Manhattan,Hamilton Heights,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Sandwich Place,Indian Restaurant,Sushi Restaurant,Park,Deli / Bodega,Yoga Studio
7,Manhattan,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega,Taco Place,Gym,Grocery Store,Beer Bar,Liquor Store
20,Manhattan,Lower East Side,Chinese Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,Cocktail Bar,Bakery,Ramen Restaurant,Art Gallery,Japanese Restaurant
25,Manhattan,Manhattan Valley,Pizza Place,Bar,Mexican Restaurant,Indian Restaurant,Coffee Shop,Playground,Thai Restaurant,Yoga Studio,Korean Restaurant,Hostel
36,Manhattan,Tudor City,Park,Mexican Restaurant,Café,Deli / Bodega,Diner,Pizza Place,Coffee Shop,Asian Restaurant,Greek Restaurant,Gym


#### Cluster 2

In [68]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[0,1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Department Store
1,Manhattan,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Bakery,Hotpot Restaurant,Optical Shop,Vietnamese Restaurant,Salon / Barbershop,Dessert Shop
5,Manhattan,Manhattanville,Coffee Shop,Seafood Restaurant,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Park,Deli / Bodega,Supermarket,Boutique,Spanish Restaurant
6,Manhattan,Central Harlem,African Restaurant,Bar,French Restaurant,American Restaurant,Seafood Restaurant,Chinese Restaurant,Caribbean Restaurant,Spa,Dessert Shop,Beer Bar
8,Manhattan,Upper East Side,Italian Restaurant,Coffee Shop,Exhibit,Art Gallery,Bakery,Gym / Fitness Center,Juice Bar,Pizza Place,French Restaurant,Yoga Studio
9,Manhattan,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Diner,Mexican Restaurant,Sushi Restaurant,Japanese Restaurant
10,Manhattan,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Burger Joint,Café,Gym,Gym / Fitness Center,Mexican Restaurant
12,Manhattan,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Indian Restaurant,Bakery,Café,Mediterranean Restaurant,Restaurant,Pub
15,Manhattan,Midtown,Hotel,Coffee Shop,Clothing Store,Bookstore,American Restaurant,Theater,Bakery,Sporting Goods Shop,Café,French Restaurant
16,Manhattan,Murray Hill,Sandwich Place,Coffee Shop,American Restaurant,Japanese Restaurant,Italian Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Hotel,Gym,Chinese Restaurant


#### Cluster 3

In [69]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[0,1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Manhattan,Lincoln Square,Plaza,Theater,Italian Restaurant,Café,Performing Arts Venue,Concert Hall,Gym,French Restaurant,American Restaurant,Indie Movie Theater
14,Manhattan,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,American Restaurant,Coffee Shop,Spa,Hotel,Wine Shop,Gym,Sandwich Place
21,Manhattan,Tribeca,American Restaurant,Park,Italian Restaurant,Café,Spa,Wine Bar,Men's Store,Greek Restaurant,Coffee Shop,Hotel
24,Manhattan,West Village,Italian Restaurant,New American Restaurant,American Restaurant,Jazz Club,Cocktail Bar,Park,Wine Bar,Bakery,Ice Cream Shop,Theater
39,Manhattan,Hudson Yards,American Restaurant,Italian Restaurant,Hotel,Gym / Fitness Center,Coffee Shop,Café,Thai Restaurant,Asian Restaurant,Dog Run,Spanish Restaurant


#### Cluster 4

In [70]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[0,1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Manhattan,Stuyvesant Town,Bar,Park,Baseball Field,Heliport,Gas Station,German Restaurant,Boat or Ferry,Farmers Market,Bistro,Gym / Fitness Center


#### Cluster 5

In [71]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[0,1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Manhattan,Roosevelt Island,Sandwich Place,Coffee Shop,Gym,Park,Soccer Field,Farmers Market,Supermarket,Metro Station,School,Outdoors & Recreation
